In [6]:
#First install the necessary packages
!pip install -q -q -q trl==0.16.0
!pip install -q -q -q peft==0.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
#Import the required dependencies for this project
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig

from datasets import Dataset, load_dataset
from collections import defaultdict
from evaluate import load
import random

In [8]:
# load the entire dataset
ds = load_dataset('bitext/Bitext-travel-llm-chatbot-training-dataset', split="train")

#Check how many rows are available in the dataset
print(len(ds))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.21k [00:00<?, ?B/s]

(…)-travel-llm-chatbot-training-dataset.csv:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31658 [00:00<?, ? examples/s]

31658


In [9]:
#Train/Test Split
split_dataset = ds.train_test_split(test_size=0.2, seed=42)
train_ds = split_dataset['train']
eval_ds = split_dataset['test']

# Group examples by intent
random.seed(42)
intent_groups = defaultdict(list)
for record in train_ds:
    intent = record["intent"]
    intent_groups[intent].append(record)

# Determine how many samples per intent
total_intents = len(intent_groups)
total_num_of_records = 5000
samples_per_intent = total_num_of_records // total_intents

#balanced dataset
balanced_subset = []
for intent, examples in intent_groups.items():
    sampled = random.sample(examples, min(samples_per_intent, len(examples)))
    balanced_subset.extend(sampled)

# Shuffle the final balanced_subset (prevents the model from seeing samples grouped by intent during training)
random.shuffle(balanced_subset)


travel_chat_ds = Dataset.from_list(balanced_subset)

In [10]:
# Load the Llama Model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def merge_example(row):
    row['conversation'] = f"Query: {row['instruction']}\nIntent: {row['intent']}\nResponse: {row['response']}"
    return row

travel_chat_ds = travel_chat_ds.map(merge_example)

config.json:   0%|          | 0.00/652 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Map:   0%|          | 0/4983 [00:00<?, ? examples/s]

In [11]:
# Initialize LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout = 0.05,
    bias = "none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj','v_proj']
)


# Initialize SFTConfig
sftConfig = SFTConfig(
    num_train_epochs = 3,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 2,
    learning_rate = 2e-4,
    max_grad_norm = 0.3,
    save_steps = 500,
    dataset_text_field = 'conversation',
    report_to = "none",
)

In [ ]:
'''Since my focus was to ensure the model trained correctly and to validate the logic of my implementation,
   I chose to disable logging for simplicity and efficiency.'''

import os
os.environ["WANDB_DISABLED"] = "true"

# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=travel_chat_ds,
    peft_config=lora_config,
    args=sftConfig
)

trainer.train()

Converting train dataset to ChatML:   0%|          | 0/4983 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/4983 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/4983 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/4983 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


In [ ]:
rouge = load("rouge")

eval_ds = eval_ds.map(merge_example)

def generate_predictions(model, tokenizer, dataset, max_length=200):
    predictions = []
    references = []

    for example in dataset:
        input_text = f"Query: {example['instruction']}\nIntent: {example['intent']}\nResponse:"
        inputs = tokenizer(input_text, return_tensors="pt").to(model.device) #moving the input tensors to the same device as the model

        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id
        )
        # attention mask to tell the model to ignore the padding since the model don't know the padding

        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        pred_response = pred.split("Response:")[-1].strip()
        predictions.append(pred_response)
        references.append(example['response'])

    return predictions, references

trainer.model.eval()
# Generate predictions and references
predictions, references = generate_predictions(trainer.model, tokenizer, eval_ds)

# Calculate ROUGE scores
rouge_scores = rouge.compute(
    predictions=predictions,
    references=references,
    use_stemmer=True
)
# Stemming reduces words to their root or base form by chopping off suffixes. traveling->travel

# Print ROUGE scores
print("ROUGE Scores:")
print(f"ROUGE-1: {rouge_scores['rouge1'].mid}")
print(f"ROUGE-2: {rouge_scores['rouge2'].mid}")
print(f"ROUGE-L: {rouge_scores['rougeL'].mid}")
print(f"ROUGE-Lsum: {rouge_scores['rougeLsum'].mid}")

In [ ]:
# Generate responses with fine-tuned model
inputs = tokenizer.encode("Query: where can i find my baggage", return_tensors="pt")
outputs = trainer.model.generate(inputs, max_new_tokens=25)
decoded_outputs = tokenizer.decode(outputs[0, inputs.shape[1]:], skip_special_tokens=True)
model_response = decoded_outputs
print(model_response)